In [45]:
import sys
sys.dont_write_bytecode = True
import pandas as pd
# import kplot as kp

In [ ]:
# from pyecharts.charts import Kline
from pyecharts.charts import Kline, Bar, Grid
from pyecharts import options as opts
import akshare as ak
import pandas as pd

# from pyecharts.globals import CurrentConfig,NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from typing import List, Tuple, Dict, Any, Union, Optional


def trans_aksdat_to_echartdat(df_data: pd.DataFrame) -> Tuple[List, List, List, List]:
    """ """
    kline_data = df_data[["开盘", "收盘", "最低", "最高"]].values.tolist()
    date = df_data["日期"].values.tolist()
    volumn_data = df_data[["成交量"]].values.tolist()
    value_data = df_data[["成交额"]].values.tolist()
    return date, kline_data, volumn_data, value_data


def draw_kline(
    date, kline_data, yaxis_name="Price", title="Stock K-Line Chart", markline=None
) -> Kline:
    """ """
    kline = (
        Kline()
        .add_xaxis(date)
        .add_yaxis(
            yaxis_name,
            y_axis=kline_data,
            markline_opts=opts.MarkLineOpts(
                data=(
                    [
                        opts.MarkLineItem(type_="max", name="Max"),
                        opts.MarkLineItem(type_="min", name="Min"),
                    ]
                    if markline
                    else None
                ),
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                is_scale=True,
            ),
            datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        )
    )
    return kline


def draw_bar(date, volumn_data, yaxis_name="Volume", title="Stock Volume Chart") -> Bar:
    bar = (
        Bar().add_xaxis(date).add_yaxis(yaxis_name, y_axis=volumn_data)
    ).set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
        yaxis_opts=opts.AxisOpts(type_="value", is_scale=True),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="vertical"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
    )

    return bar


class kchartDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def getData(self, symbol, period="daily", adjust="qfq", **kwargs):
        self[symbol] = ak.stock_zh_a_hist(symbol, period, adjust, **kwargs)
        return self[symbol]

    def plot_aks(self, symbol, period="daily", adjust="qfq", **kwargs):
        if symbol not in self.keys():
            self.getData(symbol, period, adjust, **kwargs)
        date, kline_data, volumn_data, value_data = trans_aksdat_to_echartdat(
            self[symbol]
        )

        kline = draw_kline(
            date, kline_data, symbol, title="stock " + symbol + " Kline Chart"
        )

        volumn_bar = draw_bar(
            date,
            volumn_data,
        )

        grid = (
            Grid(init_opts=opts.InitOpts(width="900px", height="600px"))
            .add(
                kline,
                grid_opts=opts.GridOpts(pos_bottom="30%"),
                is_control_axis_index=True,
            )
            .add(
                volumn_bar,
                grid_opts=opts.GridOpts(pos_top="70%"),
            )
        )
        grid.render()
        return grid

In [98]:
kchart = kchartDict()


In [99]:
kchart.plot_aks(symbol='000002',period='monthly')
